In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv('data.csv',usecols=["Stock name","Quantity","Buy date","Buy price","Buy value","Sell date","Sell price","Sell value","Realised P&L"])
print(df.columns)

Index(['Stock name', 'Quantity', 'Buy date', 'Buy price', 'Buy value',
       'Sell date', 'Sell price', 'Sell value', 'Realised P&L'],
      dtype='object')


In [3]:
df.head()

,Stock name,Quantity,Buy date,Buy price,Buy value,Sell date,Sell price,Sell value,Realised P&L
0,RITES LIMITED,2,04-06-2024,635.0,1270.0,14-06-2024,705.0,1410.0,140.0
1,RAIL VIKAS NIGAM LIMITED,10,04-06-2024,350.0,3500.0,21-06-2024,405.0,4050.0,550.0
2,INDIAN RAILWAY FIN CORP L,5,04-06-2024,174.0,870.0,21-06-2024,179.0,895.0,25.0
3,ORIENT BELL LIMITED,5,04-06-2024,340.0,1700.0,24-06-2024,383.0,1915.0,215.0
4,ONE 97 COMMUNICATIONS LTD,2,04-06-2024,357.4,714.8,26-06-2024,408.0,816.0,101.2


In [4]:
df['Buy date'] = pd.to_datetime(df['Buy date'], dayfirst=True, errors='coerce')
df['Sell date'] = pd.to_datetime(df['Sell date'], dayfirst=True, errors='coerce')
df.head()

,Stock name,Quantity,Buy date,Buy price,Buy value,Sell date,Sell price,Sell value,Realised P&L
0,RITES LIMITED,2,2024-06-04,635.0,1270.0,2024-06-14,705.0,1410.0,140.0
1,RAIL VIKAS NIGAM LIMITED,10,2024-06-04,350.0,3500.0,2024-06-21,405.0,4050.0,550.0
2,INDIAN RAILWAY FIN CORP L,5,2024-06-04,174.0,870.0,2024-06-21,179.0,895.0,25.0
3,ORIENT BELL LIMITED,5,2024-06-04,340.0,1700.0,2024-06-24,383.0,1915.0,215.0
4,ONE 97 COMMUNICATIONS LTD,2,2024-06-04,357.4,714.8,2024-06-26,408.0,816.0,101.2


In [5]:
# Filter only sold trades (Sell date not NaN)
df = df.dropna(subset=['Sell date'])

In [6]:
df['Holding_Days'] = (df['Sell date'] - df['Buy date']).dt.days

# Tag if STCG or LTCG
df['Type'] = df['Holding_Days'].apply(lambda x: 'STCG' if x < 365 else 'LTCG')

In [7]:
# Split STCG before/after 23 July 2024
split_date = datetime(2024, 7, 23)
def applySplitTag(row:pd.DataFrame):
    if row["Type"]=="STCG" and row["Sell date"]<split_date:
        return "Before_23July"
    elif row["Type"]=="STCG" and row["Sell date"]>=split_date:
        return "After_23July"
    else:
        return None
df['STCG_Period'] = df.apply(applySplitTag,axis=1)

In [8]:
# Summary for Schedule CG main table
summary = {
    'STCG_Before_23July': {
        'Sell_Value': df[df['STCG_Period'] == 'Before_23July']['Sell value'].sum(),
        'Buy_Value': df[df['STCG_Period'] == 'Before_23July']['Buy value'].sum(),
        'Gain': df[df['STCG_Period'] == 'Before_23July']['Realised P&L'].sum()
    },
    'STCG_After_23July': {
        'Sell_Value': df[df['STCG_Period'] == 'After_23July']['Sell value'].sum(),
        'Buy_Value': df[df['STCG_Period'] == 'After_23July']['Buy value'].sum(),
        'Gain': df[df['STCG_Period'] == 'After_23July']['Realised P&L'].sum()
    },
    'LTCG': {
        'Sell_Value': df[df['Type'] == 'LTCG']['Sell value'].sum(),
        'Buy_Value': df[df['Type'] == 'LTCG']['Buy value'].sum(),
        'Gain': df[df['Type'] == 'LTCG']['Realised P&L'].sum()
    }
}
summary

{'STCG_Before_23July': {'Sell_Value': np.float64(48997.45),
  'Buy_Value': np.float64(44225.549999999996),
  'Gain': np.float64(4771.9)},
 'STCG_After_23July': {'Sell_Value': np.float64(48699.05),
  'Buy_Value': np.float64(45264.31),
  'Gain': np.float64(3434.74)},
 'LTCG': {'Sell_Value': np.float64(0.0),
  'Buy_Value': np.float64(0.0),
  'Gain': np.float64(0.0)}}

In [9]:
# Table F logic (Advance Tax Quarters) → **Only Gains**
def get_period(date):
    if date <= datetime(2024, 6, 15):
        return 'Upto 15/6'
    elif date <= datetime(2024, 9, 15):
        return '16/6-15/9'
    elif date <= datetime(2024, 12, 15):
        return '16/9-15/12'
    elif date <= datetime(2025, 3, 15):
        return '16/12-15/3'
    else:
        return '16/3-31/3'

df['Tax_Period'] = df['Sell date'].apply(get_period)

In [10]:
# Group STCG before/after July for Table F (Gains)
tableF_15 = df[df['STCG_Period'] == 'Before_23July'].groupby('Tax_Period')['Realised P&L'].sum().to_dict()
tableF_20 = df[df['STCG_Period'] == 'After_23July'].groupby('Tax_Period')['Realised P&L'].sum().to_dict()

In [11]:
# Final Output
print("\n=== Summary for Schedule CG ===")
print(summary)
print("\n=== Table F (15%) ===")
print(tableF_15)
print("\n=== Table F (20%) ===")
print(tableF_20)

# Totals for BFLA validation
print("\n=== Totals for BFLA ===")
print(f"15% Gain Total: {summary['STCG_Before_23July']['Gain']}")
print(f"20% Gain Total: {summary['STCG_After_23July']['Gain']}")
print(f"LTCG Total: {summary['LTCG']['Gain']}")


=== Summary for Schedule CG ===
{'STCG_Before_23July': {'Sell_Value': np.float64(48997.45), 'Buy_Value': np.float64(44225.549999999996), 'Gain': np.float64(4771.9)}, 'STCG_After_23July': {'Sell_Value': np.float64(48699.05), 'Buy_Value': np.float64(45264.31), 'Gain': np.float64(3434.74)}, 'LTCG': {'Sell_Value': np.float64(0.0), 'Buy_Value': np.float64(0.0), 'Gain': np.float64(0.0)}}

=== Table F (15%) ===
{'16/6-15/9': 3424.95, 'Upto 15/6': 1346.95}

=== Table F (20%) ===
{'16/6-15/9': 2493.55, '16/9-15/12': 941.19}

=== Totals for BFLA ===
15% Gain Total: 4771.9
20% Gain Total: 3434.74
LTCG Total: 0.0
